In [ ]:
import itertools
import importlib

import numpy as np
import networkx as nx

import torch
from torch_geometric import utils as gutils

from awe import features, graph_utils, awe_graph, filtering
from awe.data import swde, dataset

for module in [features, graph_utils, awe_graph, filtering, dataset, swde]:
    importlib.reload(module)

In [ ]:
sds = swde.Dataset(suffix='-exact')
ds = dataset.DatasetCollection()
#ds.node_predicate = filtering.LeafNodePredicate()

In [ ]:
import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting
import bokeh.transform
bokeh.io.output_notebook()

In [ ]:
page = sds.verticals[0].websites[3].pages[188]
page_ctx = ds.prepare_page_context(page)
page.file_path

In [ ]:
example_node = [n for n in page_ctx.nodes if n.labels == ['price']][0]
example_node

In [ ]:
graph_utils.remove_single_nodes(page_ctx)

In [ ]:
graph = graph_utils.to_networkx(page_ctx)
graph.nodes[example_node.dataset_index]

In [ ]:
REAL_LAYOUT = True
layout = nx.get_node_attributes(graph, 'center') if REAL_LAYOUT else nx.planar_layout
bokeh_graph = bokeh.plotting.from_networkx(graph, layout)

In [ ]:
label_cmap = bokeh.transform.factor_cmap(
    'label', bokeh.palettes.Spectral5, page.fields)

def node_glyph(**kwargs):
    return bokeh.models.Circle(
        size='display_size',
        fill_color=label_cmap,
        **kwargs
    )

bokeh_graph.node_renderer.glyph = node_glyph()
bokeh_graph.node_renderer.hover_glyph = node_glyph(line_width=2)
bokeh_graph.node_renderer.selection_glyph = node_glyph(line_width=2)
bokeh_graph.edge_renderer.glyph = bokeh.models.MultiLine(line_alpha=0.5)
bokeh_graph.edge_renderer.hover_glyph = bokeh.models.MultiLine(line_width=2)
bokeh_graph.edge_renderer.selection_glyph = bokeh.models.MultiLine(line_width=2)
bokeh_graph.selection_policy = bokeh.models.NodesAndLinkedEdges()
bokeh_graph.inspection_policy = bokeh.models.NodesAndLinkedEdges()

In [ ]:
TOOLTIPS = [
    ('index', '$index'),
    ('size', '@display_size'),
    ('summary', '@summary'),
    ('label', '@label')
]

plot = bokeh.plotting.figure(
    tooltips=TOOLTIPS,
    active_scroll='wheel_zoom'
)

plot.renderers.append(bokeh_graph)
plot.y_range.flipped = True
bokeh.io.show(plot)